In [1]:
from __future__ import annotations

from uuid import uuid4
from datetime import datetime

from zep_python.models import Session
from dotenv import load_dotenv
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ZepMemory
from langchain.prompts.prompt import PromptTemplate
from langchain.schema import BaseMessage
from langchain.input import get_colored_text
from langchain.schema import PromptValue
from langchain.callbacks.manager import CallbackManagerForChainRun
from typing import Any, Optional, Dict, List, Tuple

In [2]:
ZEP_API_URL = "http://localhost:8000"

session_id = str(uuid4())
memory = ZepMemory(
    session_id=session_id,
    url=ZEP_API_URL,
    memory_key="chat_history",
    input_key="input",
)

In [3]:
load_dotenv()

True

In [4]:
def get_entities_from_memory(
    memory: ZepMemory,
    include_role: list[str] | None = None,
    include_label: list[str] | None = None,
    exclude_label: list[str] | None = None,
) -> str:
    if include_role is None:
        include_role = ["human", "ai"]

    entities_strs = []
    for message in memory.chat_memory.zep_messages:
        if message.role not in include_role:
            continue

        if not (message.metadata and message.metadata.get("system").get("entities")):
            continue

        entities_str_part = []
        for entity in message.metadata.get("system").get("entities"):
            if (
                include_label is not None and entity.get("Label") not in include_label
            ) or (exclude_label is not None and entity.get("Label") in exclude_label):
                continue

            entities_str_part.append(f"{entity.get('Name')} ({entity.get('Label')})")

        if entities_str_part:
            entities_strs.append(f"{message.role}: {', '.join(entities_str_part)}")

    return "\n".join(entities_strs)

In [5]:
prompt_template = """You are a conversational bot who enjoys learning new things about
the people you chat with. You're very curious about the world and want to know more about it.

Here are some of the things that have been discussed in the past:
{entities}

Here are the prior messages in this conversation:
{chat_history}

Here is a question: {input}
"""

prompt = PromptTemplate(
    template=prompt_template, input_variables=["input", "entities", "chat_history"]
)

In [6]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

chain = LLMChain(llm=llm, memory=memory, prompt=prompt, verbose=True)

In [7]:
print(
    chain.run(
        {
            "input": "Hello! My name is Mary! I'm visiting Maui next week. What should I do there?",
            "entities": get_entities_from_memory(
                memory, include_label=["GPE", "FAC", "LOC"]
            ),
        }
    )
)

Session cd03b41c-8deb-4a91-868d-4af905f3f9e3 not found in Zep. Returning None
Session cd03b41c-8deb-4a91-868d-4af905f3f9e3 not found in Zep. Returning None




> Entering new LLMChain chain...
Prompt after formatting:
You are a conversational bot who enjoys learning new things about
the people you chat with. You're very curious about the world and want to know more about it.

Here are some of the things that have been discussed in the past:


Here are the prior messages in this conversation:


Here is a question: Hello! My name is Mary! I'm visiting Maui next week. What should I do there?


> Finished chain.
Hello Mary! How exciting that you're visiting Maui! There are plenty of amazing things to do on the island. Here are a few suggestions:

1. Explore the Road to Hana: This scenic drive takes you through lush rainforests, beautiful waterfalls, and stunning coastal views. Make sure to stop at the black sand beach, Wai'anapanapa State Park.

2. Visit Haleakalā National Park: This park is home to the Haleakalā volcano and offers breathtaking sunrise and sunset views. You can also go hiking or biking through the unique landscape.

3. Relax at

In [8]:
print(
    chain.run(
        {
            "input": "I'm definitely looking forward to the beach and food!",
            "entities": get_entities_from_memory(
                memory, include_label=["GPE", "FAC", "LOC"]
            ),
        }
    )
)



> Entering new LLMChain chain...
Prompt after formatting:
You are a conversational bot who enjoys learning new things about
the people you chat with. You're very curious about the world and want to know more about it.

Here are some of the things that have been discussed in the past:


Here are the prior messages in this conversation:
Human: Hello! My name is Mary! I'm visiting Maui next week. What should I do there?
AI: Hello Mary! How exciting that you're visiting Maui! There are plenty of amazing things to do on the island. Here are a few suggestions:

1. Explore the Road to Hana: This scenic drive takes you through lush rainforests, beautiful waterfalls, and stunning coastal views. Make sure to stop at the black sand beach, Wai'anapanapa State Park.

2. Visit Haleakalā National Park: This park is home to the Haleakalā volcano and offers breathtaking sunrise and sunset views. You can also go hiking or biking through the unique landscape.

3. Relax at the beautiful beaches: Maui ha

In [9]:
print(
    chain.run(
        {
            "input": "Thank you! Do you have recommendations for hiking on the island?",
            "entities": get_entities_from_memory(
                memory, include_label=["GPE", "FAC", "LOC"]
            ),
        }
    )
)



> Entering new LLMChain chain...
Prompt after formatting:
You are a conversational bot who enjoys learning new things about
the people you chat with. You're very curious about the world and want to know more about it.

Here are some of the things that have been discussed in the past:
human: Maui (LOC)
ai: Maui (LOC), State Park (FAC), Ka'anapali Beach (GPE), Wailea Beach (GPE), Makena Beach (GPE)

Here are the prior messages in this conversation:
Human: Hello! My name is Mary! I'm visiting Maui next week. What should I do there?
AI: Hello Mary! How exciting that you're visiting Maui! There are plenty of amazing things to do on the island. Here are a few suggestions:

1. Explore the Road to Hana: This scenic drive takes you through lush rainforests, beautiful waterfalls, and stunning coastal views. Make sure to stop at the black sand beach, Wai'anapanapa State Park.

2. Visit Haleakalā National Park: This park is home to the Haleakalā volcano and offers breathtaking sunrise and sunset

In [10]:
print(
    chain.run(
        {
            "input": "Please list all the places you've recommended so far.",
            "entities": get_entities_from_memory(
                memory, include_label=["GPE", "FAC", "LOC"]
            ),
        }
    )
)



> Entering new LLMChain chain...
Prompt after formatting:
You are a conversational bot who enjoys learning new things about
the people you chat with. You're very curious about the world and want to know more about it.

Here are some of the things that have been discussed in the past:
human: Maui (LOC)
ai: Maui (LOC), State Park (FAC), Ka'anapali Beach (GPE), Wailea Beach (GPE), Makena Beach (GPE)
ai: Maui (LOC)

Here are the prior messages in this conversation:
Human: Hello! My name is Mary! I'm visiting Maui next week. What should I do there?
AI: Hello Mary! How exciting that you're visiting Maui! There are plenty of amazing things to do on the island. Here are a few suggestions:

1. Explore the Road to Hana: This scenic drive takes you through lush rainforests, beautiful waterfalls, and stunning coastal views. Make sure to stop at the black sand beach, Wai'anapanapa State Park.

2. Visit Haleakalā National Park: This park is home to the Haleakalā volcano and offers breathtaking sun

In [11]:
memory.chat_memory.messages

[HumanMessage(content="Hello! My name is Mary! I'm visiting Maui next week. What should I do there?", additional_kwargs={'uuid': 'e7cfa0d1-90d9-49c0-a55d-889c8db4fa36', 'created_at': '2023-07-17T00:42:29.39709Z', 'token_count': 22, 'metadata': {'system': {'entities': [{'Label': 'PERSON', 'Matches': [{'End': 22, 'Start': 18, 'Text': 'Mary'}], 'Name': 'Mary'}, {'Label': 'LOC', 'Matches': [{'End': 41, 'Start': 37, 'Text': 'Maui'}], 'Name': 'Maui'}, {'Label': 'DATE', 'Matches': [{'End': 51, 'Start': 42, 'Text': 'next week'}], 'Name': 'Next week'}], 'intent': 'The subject is seeking suggestions or recommendations for activities or attractions to do in Maui.'}}}, example=False),
 AIMessage(content="Hello Mary! How exciting that you're visiting Maui! There are plenty of amazing things to do on the island. Here are a few suggestions:\n\n1. Explore the Road to Hana: This scenic drive takes you through lush rainforests, beautiful waterfalls, and stunning coastal views. Make sure to stop at the b

In [12]:
# Persist a snapshot of location entities to the Session metadata
def get_location_entities(
    memory: ZepMemory,
) -> dict[str, dict]:
    entities = {}
    index = 0
    for message in memory.chat_memory.zep_messages:
        if not (message.metadata and message.metadata.get("system").get("entities")):
            continue

        for entity in message.metadata.get("system").get("entities"):
            if entity.get("Label") not in ["GPE", "FAC", "LOC"]:
                continue
            if message.role not in entities:
                entities[message.role] = {}

            entities[message.role][index] = {
                "created_at": message.created_at,
                "name": entity.get("Name"),
                "label": entity.get("Label"),
            }
            index += 1

    return entities


# Retrieve client from ZepMemory and upsert the Session metadata
client = memory.chat_memory.zep_client
client.add_session(
    Session(
        session_id=session_id,
        metadata={
            "location_entities": get_location_entities(memory),
        },
    )
)

# Retrieve the Session metadata
client.get_session(session_id=session_id).metadata

{'location_entities': {'ai': {'1': {'created_at': '2023-07-17T00:42:29.404063Z',
    'label': 'LOC',
    'name': 'Maui'},
   '10': {'created_at': '2023-07-17T00:42:44.773948Z',
    'label': 'LOC',
    'name': 'the Waihee Valley'},
   '2': {'created_at': '2023-07-17T00:42:29.404063Z',
    'label': 'FAC',
    'name': 'State Park'},
   '3': {'created_at': '2023-07-17T00:42:29.404063Z',
    'label': 'GPE',
    'name': "Ka'anapali Beach"},
   '4': {'created_at': '2023-07-17T00:42:29.404063Z',
    'label': 'GPE',
    'name': 'Wailea Beach'},
   '5': {'created_at': '2023-07-17T00:42:29.404063Z',
    'label': 'GPE',
    'name': 'Makena Beach'},
   '6': {'created_at': '2023-07-17T00:42:32.329542Z',
    'label': 'LOC',
    'name': 'Maui'},
   '7': {'created_at': '2023-07-17T00:42:44.773948Z',
    'label': 'LOC',
    'name': 'Maui'},
   '8': {'created_at': '2023-07-17T00:42:44.773948Z',
    'label': 'FAC',
    'name': 'the Pipiwai Trail'},
   '9': {'created_at': '2023-07-17T00:42:44.773948Z',
   

In [13]:
prompt_template = """You are a conversational bot who enjoys learning new things
about the people you chat with. You're very curious about the world and want to know more about it.

If you're asked about dates, here is the current date and time is {date}.

Here are the prior messages in this conversation:
{chat_history}

Here is the human's question: {input}
"""

prompt = PromptTemplate(
    template=prompt_template, input_variables=["input", "date", "chat_history"]
)


class EnhancedMemChain(LLMChain):
    memory_key: str = "chat_history"

    def _get_chat_history(self, chat_history: list[BaseMessage]) -> str:
        buffer = ""
        for dialogue_turn in chat_history:
            timestamp = dialogue_turn.additional_kwargs.get("created_at")
            buffer += f"\n{timestamp} | {dialogue_turn.type}: {dialogue_turn.content}"

        return buffer

    def prep_prompts(
        self,
        input_list: List[Dict[str, Any]],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Tuple[List[PromptValue], Optional[List[str]]]:
        """Prepare prompts from inputs."""

        stop = None
        if "stop" in input_list[0]:
            stop = input_list[0]["stop"]
        prompts = []
        for inputs in input_list:
            chat_history: List[BaseMessage] = inputs.get("chat_history")
            selected_inputs = {k: inputs[k] for k in self.prompt.input_variables}
            if chat_history:
                chat_history_str = self._get_chat_history(chat_history)
                selected_inputs[self.memory_key] = chat_history_str
            prompt = self.prompt.format_prompt(**selected_inputs)
            _colored_text = get_colored_text(prompt.to_string(), "green")
            _text = "Prompt after formatting:\n" + _colored_text
            if run_manager:
                run_manager.on_text(_text, end="\n", verbose=self.verbose)
            if "stop" in inputs and inputs["stop"] != stop:
                raise ValueError(
                    "If `stop` is present in any inputs, should be present in all."
                )
            prompts.append(prompt)
        return prompts, stop

In [14]:
memory = ZepMemory(
    session_id=str(uuid4()),
    url=ZEP_API_URL,
    memory_key="chat_history",
    input_key="input",
    return_messages=True,  # Set to True to return Messages and not a string
)

llm = ChatOpenAI(model="gpt-4")

chain = EnhancedMemChain(llm=llm, memory=memory, prompt=prompt, verbose=True)

In [15]:
print(
    chain.run(
        {
            "input": "Hello! My name is Mary! I'm visiting Hawaii next Monday. What should I do there?",
            "date": datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S"),
        }
    )
)

Session 08db59c9-3039-46c7-8fe0-26b4c1cb34ef not found in Zep. Returning None




> Entering new EnhancedMemChain chain...
Prompt after formatting:
You are a conversational bot who enjoys learning new things
about the people you chat with. You're very curious about the world and want to know more about it.

If you're asked about dates, here is the current date and time is 2023-07-17 00:42:49.

Here are the prior messages in this conversation:
[]

Here is the human's question: Hello! My name is Mary! I'm visiting Hawaii next Monday. What should I do there?


> Finished chain.
Hello, Mary! It's nice to meet you. That sounds like an exciting trip! Hawaii is known for its beautiful beaches, surfing, hiking trails, and rich cultural history.

1. **Beaches:** You should definitely visit Hawaii's stunning beaches. Some of the most popular ones include Waikiki Beach in Oahu, Poipu Beach in Kauai, and Hanauma Bay in Oahu.

2. **Surfing:** If you're into water sports, you might want to try surfing. There are plenty of surf schools that welcome beginners.

3. **Hiking:** For

In [16]:
print(
    chain.run(
        {
            "input": "I heard there are helicopter tours. Do you know if they are worth it?",
            "date": datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S"),
        }
    )
)



> Entering new EnhancedMemChain chain...
Prompt after formatting:
You are a conversational bot who enjoys learning new things
about the people you chat with. You're very curious about the world and want to know more about it.

If you're asked about dates, here is the current date and time is 2023-07-17 00:43:03.

Here are the prior messages in this conversation:

2023-07-17T00:43:03.906736Z | human: Hello! My name is Mary! I'm visiting Hawaii next Monday. What should I do there?
2023-07-17T00:43:03.906821Z | ai: Hello, Mary! It's nice to meet you. That sounds like an exciting trip! Hawaii is known for its beautiful beaches, surfing, hiking trails, and rich cultural history.

1. **Beaches:** You should definitely visit Hawaii's stunning beaches. Some of the most popular ones include Waikiki Beach in Oahu, Poipu Beach in Kauai, and Hanauma Bay in Oahu.

2. **Surfing:** If you're into water sports, you might want to try surfing. There are plenty of surf schools that welcome beginners.



In [19]:
print(
    chain.run(
        {
            "input": "How long ago did we last chat?",
            "date": datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S"),
        }
    )
)



> Entering new EnhancedMemChain chain...
Prompt after formatting:
You are a conversational bot who enjoys learning new things
about the people you chat with. You're very curious about the world and want to know more about it.

If you're asked about dates, here is the current date and time is 2023-07-17 00:44:34.

Here are the prior messages in this conversation:

2023-07-17T00:43:03.906736Z | human: Hello! My name is Mary! I'm visiting Hawaii next Monday. What should I do there?
2023-07-17T00:43:03.939382Z | ai: Hello, Mary! It's nice to meet you. That sounds like an exciting trip! Hawaii is known for its beautiful beaches, surfing, hiking trails, and rich cultural history.

1. **Beaches:** You should definitely visit Hawaii's stunning beaches. Some of the most popular ones include Waikiki Beach in Oahu, Poipu Beach in Kauai, and Hanauma Bay in Oahu.

2. **Surfing:** If you're into water sports, you might want to try surfing. There are plenty of surf schools that welcome beginners.



In [18]:
print(
    chain.run(
        {
            "input": "From what date will I be visiting Hawaii?",
            "date": datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S"),
        }
    )
)



> Entering new EnhancedMemChain chain...
Prompt after formatting:
You are a conversational bot who enjoys learning new things
about the people you chat with. You're very curious about the world and want to know more about it.

If you're asked about dates, here is the current date and time is 2023-07-17 00:43:15.

Here are the prior messages in this conversation:

2023-07-17T00:43:03.906736Z | human: Hello! My name is Mary! I'm visiting Hawaii next Monday. What should I do there?
2023-07-17T00:43:03.939382Z | ai: Hello, Mary! It's nice to meet you. That sounds like an exciting trip! Hawaii is known for its beautiful beaches, surfing, hiking trails, and rich cultural history.

1. **Beaches:** You should definitely visit Hawaii's stunning beaches. Some of the most popular ones include Waikiki Beach in Oahu, Poipu Beach in Kauai, and Hanauma Bay in Oahu.

2. **Surfing:** If you're into water sports, you might want to try surfing. There are plenty of surf schools that welcome beginners.

